In [1]:
import random
import string
import re
import pickle 
from unicodedata import normalize
import numpy
from numpy import array
from numpy.random import rand
from numpy.random import shuffle
from numpy import argmax
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import SimpleRNN,GRU,LSTM
from keras.layers import Dense, Embedding, RepeatVector, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_accuracy
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Fetching and decompressing the dataset 
!!curl -O http://www.manythings.org/anki/fra-eng.zip
!!unzip fra-eng.zip

['Archive:  fra-eng.zip',
 'replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL',
 '(EOF or read error, treating as "[N]one" ...)']

In [3]:
# This function performs two tasks:

# 1. Loading the text data preserving the Unicode french characters and then
# 2. Each line of the text file contain English sentence and its French translation seperated by tab character. 

def loading_and_pairs(file):
    
	# opening the text file in read only mode with unicode encoding
	f = open(file = file, mode = 'rt', encoding = 'utf-8')
    
	# reading the text from the opened file
	text = f.read()
    
	# finally closing the file
	f.close()
    
    # Obtaining each line in the file
	l = text.strip().split('\n')
    
    # Obtaining the pairs of English sentence and its french translation
	pairs = [l_each.split('\t') for l_each in  l]
    
	return pairs   

In [4]:
# Cleaning the lines by removing all the non-printable characters, punctuation characters
# Given a list of lines cleaning them

def pairs_clean(lines_from_text):
    
	new_cleaned = list()
    
	# using regular expression for removing non-printable characters
	regular_non_print = re.compile('[^%s]' % re.escape(string.printable))
    
	# using regular expression for removing punctuation characters and obtaining translation table
	table = str.maketrans('', '', string.punctuation)
    
	for pair in lines_from_text:
        
		clean_pair = list()
		for new_line in pair:
            
			# normalization to remove canonical and compatibility related issues
			new_line = normalize('NFD', new_line).encode('ascii', 'ignore')
			new_line = new_line.decode('UTF-8')
            
			# tokenizing the white space
			new_line = new_line.split()
            
			# normalizing the text to lowercase
			new_line = [word.lower() for word in new_line]
            
			# removing punctuation from each token using regular expression table 
			new_line = [word.translate(table) for word in new_line]
            
			# removing non-printable characters using the above regular expression
			new_line = [regular_non_print.sub('', w) for w in new_line]
            
			# removing the non-alphabetic tokens such as numbers
			new_line = [word for word in new_line if word.isalpha()]
            
			# store as string
			clean_pair.append(' '.join(new_line))
		new_cleaned.append(clean_pair)
	return array(new_cleaned)

In [5]:
# saving the list of clean sentences to file
def saving_data(sentences, file):
	pickle.dump( sentences, open(file = file, mode = 'wb'))

In [6]:
# loading the text dataset
text_file = 'fra.txt'
text_pairs = loading_and_pairs(text_file)

In [7]:
# cleaning the sentences
cleaned_pairs = pairs_clean(text_pairs)

In [8]:
# saving the cleaned pairs to file
saving_data(cleaned_pairs, 'eng-fre.pkl')

In [9]:
# Total number of translation sentences
print("There a total of {} pairs of tranlations in the dataset".format(cleaned_pairs.shape[0]))

There a total of 190206 pairs of tranlations in the dataset


In [10]:
# Looking at few sentences

random_sample_list = random.sample(range(0, cleaned_pairs.shape[0]), 30)
for i in random_sample_list:
	print('English : %s\nFrench : %s \n' % (cleaned_pairs[i,0], cleaned_pairs[i,1]))

English : i wish that i couldve eaten at that restaurant with you
French : jaurais aime pouvoir manger dans ce restaurant avec toi 

English : this is really hard
French : cest tres dur 

English : he has two cats
French : il a deux chats 

English : i didnt break it
French : je ne lai pas rompu 

English : i never thought id see your face again
French : je nai jamais pense que je reverrais votre visage 

English : apples are cheap today
French : les pommes sont bon marche aujourdhui 

English : my brother is now in australia
French : mon frere est maintenant en australie 

English : my cat loves toys
French : mon chat adore les jouets 

English : tom is fighting cancer
French : tom combat le cancer 

English : im pretty sure that it wont snow today
French : je suis assez certaine quil ne neigera pas aujourdhui 

English : he confessed his crime
French : il a avoue son crime 

English : i stole the idea
French : jai vole lidee 

English : i was the last to know
French : jetais le derni

In [11]:
from pickle import load
from pickle import dump
from numpy.random import rand
from numpy.random import shuffle

In [12]:
# loading the previously cleaned data 
def loading_sentences(filename):
	return pickle.load(open(filename, 'rb'))

In [13]:
# loading the raw dataset
text_dataset = loading_sentences('eng-fre.pkl')

In [14]:
# There are over 190000 pairs of sentences
# It will take long time for training and testing the model
# Hence dataset size is reduced
num_sentences = 25000 #clean_pairs.shape[0]
reduced_dataset_25000 = text_dataset[:num_sentences, :]
train_size = numpy.rint(0.7 * num_sentences)
validation_size = numpy.rint(0.1 * num_sentences)
test_size = numpy.rint(0.2 * num_sentences)

# randomly shuffling the dataset
shuffle(reduced_dataset_25000)

# spliting the reduced dataset into train, validation and test
split_1 = int(train_size)
split_2 = int(train_size+validation_size)
split_3 = int(train_size+validation_size+test_size)
train, validation, test = reduced_dataset_25000[:split_1], reduced_dataset_25000[split_1:split_2], reduced_dataset_25000[split_2:split_3]

In [15]:
train.shape, validation.shape, test.shape

((17500, 3), (2500, 3), (5000, 3))

In [16]:
# saving the reduced dataset to training, validation and testing data

saving_data(text_dataset, 'eng-fre-total.pkl')
saving_data(train, 'eng-fre-train.pkl')
saving_data(validation, 'eng-fre-validation.pkl')
saving_data(test, 'eng-fre-test.pkl')

In [17]:
# Using keras tokenize class, for mapping the words to integers needed for modeling

def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

In [18]:
# Obtaining the maximum sentence length from the list of phrases 

def max_length(input_lines):
	return max(len(each_line.split()) for each_line in input_lines)

In [19]:
# encoding and padding the input and output sequences

def encode_sequences(tokenizer, length, lines):
	# Each input and output sequence are  encoded to integers
	text_encoded_integers = tokenizer.texts_to_sequences(lines)
	# Obtained sequences are padded with 0 values at the end to make their lenght as maxmim phrase length
	padding_sequences = pad_sequences(text_encoded_integers, maxlen=length, padding='post')
	return padding_sequences

In [20]:
# The output target sequences (English sentences) has to be one hot encoded as the model will 
# predicts probability of each word in the vocabulary as output. 

def encode_output(output_sequences, vocabulary_size):
	output_list = list()
	for output_sequence in output_sequences:
		cat = to_categorical(output_sequence, num_classes=vocabulary_size)
		output_list.append(cat)
	encoded_output = array(output_list)
	encoded_output = encoded_output.reshape(output_sequences.shape[0], output_sequences.shape[1], vocabulary_size)
	return encoded_output

In [21]:
# reverse mapping an predicted sequence of integers to a words by looking up tokenizer

def reverse_mapping(output_integer, tokenizer):
	for w, i in tokenizer.word_index.items():
		if i == output_integer:
			return w
	return None

In [22]:
# mapping the sequence of integers for generating string of words

def predict_sequence(rnn_model, tokenizer, original_data):
	prediction = rnn_model.predict(original_data, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target_list = list()
	for i in integers:
		word = reverse_mapping(i, tokenizer)
		if word is None:
			break
		target_list.append(word)
	return ' '.join(target_list)

In [23]:
# Evaluating the performance of each model using BLEU score by comparing predicted result to original/expected sequences 


def model_evaluation(model_name, tokenizer_used, sources_text, raw_text_dataset):
	actual_value, predicted_value = list(), list()
	bleu_scores = []
	for i, j in enumerate(sources_text):
		# translating the encoded input text sequence
		j = j.reshape((1, j.shape[0]))
		translation = predict_sequence(model_name, eng_tokenizer, j)
		raw_target, raw_src = raw_text_dataset[i][0], raw_text_dataset[i][1]
        
        # Printing 50 French to English translations by the model
        
		if i < 50:
			print('French(Source) : %s\nTarget : %s\nPredicted : %s \n' % (raw_src, raw_target, translation))
        
        
		actual_value.append([raw_target.split()])
		predicted_value.append(translation.split())
        
    # Calculating BLEU score
	bleu_score = corpus_bleu(actual_value, predicted_value, weights=(0.25, 0.25, 0.25, 0.25))
    
	# Print BLEU score
	print('BLEU score: %f' % bleu_score)
    
	return bleu_score

In [24]:
# loading the total, train, validation and test datasets 
dataset = loading_sentences('eng-fre-total.pkl')
train = loading_sentences('eng-fre-train.pkl')
valid = loading_sentences('eng-fre-validation.pkl')
test = loading_sentences('eng-fre-test.pkl')

In [25]:
dataset.shape

(190206, 3)

In [26]:
reduced_dataset_25000.shape

(25000, 3)

In [27]:
# Building the French tokenizer
french_tokenizer = create_tokenizer(reduced_dataset_25000[:, 1])
french_maximum_length = max_length(reduced_dataset_25000[:, 1])
french_vocabulary_size = len(french_tokenizer.word_index) + 1

In [28]:
print('French Vocabulary Size:', french_vocabulary_size)
print('French Maximum Sentence Length:', french_maximum_length)

French Vocabulary Size: 8021
French Maximum Sentence Length: 12


In [29]:
# Building the English tokenizer
eng_tokenizer = create_tokenizer(reduced_dataset_25000[:, 0])
english_maximum_length = max_length(reduced_dataset_25000[:, 0])
english_vocabulary_size = len(eng_tokenizer.word_index) + 1

In [30]:
print('English Vocabulary Size:',english_vocabulary_size)
print('English Maximum Sentence Length:', english_maximum_length)

English Vocabulary Size: 3949
English Maximum Sentence Length: 5


In [31]:
# Word index of English tokenizer
eng_tokenizer.word_index

{'i': 1,
 'you': 2,
 'tom': 3,
 'it': 4,
 'a': 5,
 'is': 6,
 'im': 7,
 'he': 8,
 'youre': 9,
 'me': 10,
 'was': 11,
 'the': 12,
 'are': 13,
 'we': 14,
 'this': 15,
 'its': 16,
 'to': 17,
 'that': 18,
 'go': 19,
 'do': 20,
 'were': 21,
 'have': 22,
 'your': 23,
 'dont': 24,
 'not': 25,
 'my': 26,
 'be': 27,
 'no': 28,
 'can': 29,
 'she': 30,
 'they': 31,
 'did': 32,
 'all': 33,
 'like': 34,
 'get': 35,
 'ill': 36,
 'here': 37,
 'up': 38,
 'in': 39,
 'on': 40,
 'need': 41,
 'love': 42,
 'what': 43,
 'thats': 44,
 'him': 45,
 'how': 46,
 'theyre': 47,
 'want': 48,
 'very': 49,
 'one': 50,
 'come': 51,
 'please': 52,
 'out': 53,
 'hes': 54,
 'got': 55,
 'let': 56,
 'us': 57,
 'just': 58,
 'look': 59,
 'lets': 60,
 'cant': 61,
 'so': 62,
 'am': 63,
 'help': 64,
 'now': 65,
 'stop': 66,
 'take': 67,
 'know': 68,
 'see': 69,
 'too': 70,
 'who': 71,
 'keep': 72,
 'will': 73,
 'well': 74,
 'there': 75,
 'good': 76,
 'toms': 77,
 'for': 78,
 'try': 79,
 'has': 80,
 'stay': 81,
 'at': 82,
 'feel'

In [32]:
# Word index of French tokenizer
french_tokenizer.word_index

{'je': 1,
 'tom': 2,
 'a': 3,
 'pas': 4,
 'suis': 5,
 'est': 6,
 'vous': 7,
 'il': 8,
 'nous': 9,
 'de': 10,
 'ne': 11,
 'le': 12,
 'la': 13,
 'jai': 14,
 'cest': 15,
 'un': 16,
 'tu': 17,
 'me': 18,
 'en': 19,
 'ca': 20,
 'les': 21,
 'une': 22,
 'que': 23,
 'etes': 24,
 'elle': 25,
 'sommes': 26,
 'ce': 27,
 'estce': 28,
 'es': 29,
 'sont': 30,
 'fait': 31,
 'ils': 32,
 'qui': 33,
 'tout': 34,
 'des': 35,
 'ma': 36,
 'te': 37,
 'mon': 38,
 'bien': 39,
 'elles': 40,
 'du': 41,
 'nest': 42,
 'ici': 43,
 'ete': 44,
 'besoin': 45,
 'y': 46,
 'se': 47,
 'va': 48,
 'jaime': 49,
 'faire': 50,
 'etesvous': 51,
 'cela': 52,
 'personne': 53,
 'moi': 54,
 'tres': 55,
 'faut': 56,
 'votre': 57,
 'etait': 58,
 'lair': 59,
 'veux': 60,
 'peux': 61,
 'ai': 62,
 'fais': 63,
 'soyez': 64,
 'au': 65,
 'jetais': 66,
 'aller': 67,
 'avons': 68,
 'tous': 69,
 'ton': 70,
 'train': 71,
 'on': 72,
 'lai': 73,
 'cetait': 74,
 'sest': 75,
 'comment': 76,
 'sois': 77,
 'monde': 78,
 'maintenant': 79,
 'estu': 8

In [33]:
# Building the training dataset
x_train = encode_sequences(french_tokenizer, french_maximum_length, train[:, 1])
y_train = encode_sequences(eng_tokenizer, english_maximum_length, train[:, 0])
y_train = encode_output(y_train, english_vocabulary_size)

In [34]:
# Building the validation dataset
x_valid = encode_sequences(french_tokenizer, french_maximum_length, valid[:, 1])
y_valid = encode_sequences(eng_tokenizer, english_maximum_length, valid[:, 0])
y_valid = encode_output(y_valid, english_vocabulary_size)

In [35]:
# Building testing dataset
x_test = encode_sequences(french_tokenizer, french_maximum_length, test[:, 1])
y_test = encode_sequences(eng_tokenizer, english_maximum_length, test[:, 0])
y_test = encode_output(y_test, english_vocabulary_size)

In [36]:
# Model 1: Both Encoder GRU and Decoder GRU cell
def both_gru(french_vocab, english_vocab, french_timesteps, english_timesteps, num_units):
	learning_rate = 1e-2
	model = Sequential()
	model.add(Embedding(french_vocab, num_units, input_length = french_timesteps, mask_zero = True))
	model.add(GRU(num_units))
	model.add(RepeatVector(english_timesteps))
	model.add(GRU(num_units,return_sequences=True))
	model.add(TimeDistributed(Dense(english_vocab, activation='softmax')))
	return model

In [37]:
# Defining the Model-1

learning_rate = 1e-2
both_gru = both_gru(french_vocabulary_size, english_vocabulary_size, french_maximum_length, english_maximum_length, 256)
both_gru.compile(optimizer = Adam(learning_rate), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [38]:
# Summary of the Model-1

print(both_gru.summary())
plot_model(both_gru, to_file='gru_model.png', show_shapes=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 12, 256)           2053376   
_________________________________________________________________
gru (GRU)                    (None, 256)               394752    
_________________________________________________________________
repeat_vector (RepeatVector) (None, 5, 256)            0         
_________________________________________________________________
gru_1 (GRU)                  (None, 5, 256)            394752    
_________________________________________________________________
time_distributed (TimeDistri (None, 5, 3949)           1014893   
Total params: 3,857,773
Trainable params: 3,857,773
Non-trainable params: 0
_________________________________________________________________
None
('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.g

In [39]:
# Fitting the Model-1
filename = 'both_gru.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
history_both_gru = both_gru.fit(x_train, y_train, epochs = 30, batch_size = 64, validation_data=(x_valid, y_valid), callbacks = [checkpoint], verbose = 2)

Epoch 1/30
274/274 - 43s - loss: 3.8243 - categorical_accuracy: 0.4425 - val_loss: 3.2333 - val_categorical_accuracy: 0.5005

Epoch 00001: val_loss improved from inf to 3.23328, saving model to both_gru.h5
Epoch 2/30
274/274 - 23s - loss: 2.8783 - categorical_accuracy: 0.5361 - val_loss: 2.7792 - val_categorical_accuracy: 0.5606

Epoch 00002: val_loss improved from 3.23328 to 2.77923, saving model to both_gru.h5
Epoch 3/30
274/274 - 23s - loss: 2.3850 - categorical_accuracy: 0.5819 - val_loss: 2.5848 - val_categorical_accuracy: 0.5777

Epoch 00003: val_loss improved from 2.77923 to 2.58478, saving model to both_gru.h5
Epoch 4/30
274/274 - 23s - loss: 2.0623 - categorical_accuracy: 0.6075 - val_loss: 2.4162 - val_categorical_accuracy: 0.5967

Epoch 00004: val_loss improved from 2.58478 to 2.41619, saving model to both_gru.h5
Epoch 5/30
274/274 - 23s - loss: 1.7980 - categorical_accuracy: 0.6337 - val_loss: 2.3461 - val_categorical_accuracy: 0.6007

Epoch 00005: val_loss improved from 2.

In [40]:
# Saving the history of the Model-1

df_history_both_gru = pd.DataFrame(history_both_gru.history)
df_history_both_gru.to_csv('df_history_both_gru.csv')
print(df_history_both_gru)

        loss  categorical_accuracy  val_loss  val_categorical_accuracy
0   3.824321              0.442469  3.233278                   0.50048
1   2.878268              0.536149  2.779230                   0.56064
2   2.385043              0.581851  2.584778                   0.57768
3   2.062311              0.607543  2.416189                   0.59672
4   1.797962              0.633657  2.346069                   0.60072
5   1.592628              0.654857  2.294309                   0.61408
6   1.428137              0.675337  2.274472                   0.61616
7   1.294781              0.696229  2.271312                   0.61776
8   1.186878              0.711989  2.238282                   0.62448
9   1.077078              0.731029  2.231904                   0.62792
10  1.000781              0.745874  2.252761                   0.62720
11  0.960913              0.753394  2.276547                   0.63048
12  0.905064              0.766183  2.301702                   0.63184
13  0.

In [41]:
# Loading the Model-1
both_gru = load_model('both_gru.h5')

In [42]:
#  Model-1 on the training sequences

bleu_train_both_gru = model_evaluation(both_gru, eng_tokenizer, x_train, train)

French(Source) : appretetoi
Target : prepare yourself
Predicted : sorry yourself 

French(Source) : avezvous pris une douche
Target : have you showered
Predicted : did you showered 

French(Source) : rentre a la maison
Target : go home
Predicted : come home home 

French(Source) : ils ont tort
Target : they are wrong
Predicted : they wrong cool 

French(Source) : aidemoi a sortir
Target : help me out
Predicted : show me out 

French(Source) : prenez les commandes
Target : take command
Predicted : take command 

French(Source) : restez ou vous etes
Target : hold it
Predicted : close everyone 

French(Source) : je suis ruinee
Target : im ruined
Predicted : im ruined 

French(Source) : je ne suis pas malheureux
Target : im not unhappy
Predicted : im not impressed 

French(Source) : je peux y parvenir
Target : i can do it
Predicted : i can do it 

French(Source) : jaime tes jambes
Target : i like your legs
Predicted : i like your legs 

French(Source) : maitrisezvous
Target : control yours

In [43]:
# Testing Model-1 

bleu_test_both_gru = model_evaluation(both_gru, eng_tokenizer, x_test, test)

French(Source) : jai oublie le livre
Target : i forgot the book
Predicted : i bought the book 

French(Source) : tom est sans domicile fixe
Target : toms homeless
Predicted : tom fell 

French(Source) : jaide tom
Target : i help tom
Predicted : im helping tom 

French(Source) : jai achete un cactus
Target : i bought a cactus
Predicted : i bought a stroke 

French(Source) : cest plutot grand
Target : its quite large
Predicted : this is big 

French(Source) : ouvrez le coffre
Target : open the safe
Predicted : get the 

French(Source) : il est trop confiant
Target : hes too trusting
Predicted : hes too too 

French(Source) : il adore les jouets
Target : he loves toys
Predicted : he likes oranges 

French(Source) : estce que vous avez le temps
Target : do you have time
Predicted : do i try time 

French(Source) : jai presente mes excuses
Target : i have apologized
Predicted : i got my army 

French(Source) : il nous faut de leau
Target : we need water
Predicted : we had a water 

French(S

In [44]:
# Model 2: Both Encoder LSTM and Decoder LSTM cell
def both_lstm(french_vocab, english_vocab, french_timesteps, english_timesteps, num_units):
	learning_rate = 1e-2
	model = Sequential()
	model.add(Embedding(french_vocab, num_units, input_length= french_timesteps, mask_zero=True))
	model.add(LSTM(num_units))
	model.add(RepeatVector(english_timesteps))
	model.add(LSTM(num_units,return_sequences=True))
	model.add(TimeDistributed(Dense(english_vocab, activation='softmax')))
	return model

In [45]:
# Defining Model-2

learning_rate = 1e-2
both_lstm = both_lstm(french_vocabulary_size, english_vocabulary_size, french_maximum_length, english_maximum_length, 256)
both_lstm.compile(optimizer=Adam(learning_rate), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [46]:
# Summarising Model-2

print(both_lstm.summary())
plot_model(both_lstm, to_file='both_lstm.png', show_shapes=True)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 12, 256)           2053376   
_________________________________________________________________
lstm (LSTM)                  (None, 256)               525312    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 5, 256)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 256)            525312    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 3949)           1014893   
Total params: 4,118,893
Trainable params: 4,118,893
Non-trainable params: 0
_________________________________________________________________
None
('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz

In [47]:
# Fitting the Model-2

filename = 'both_lstm.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
history_both_lstm = both_lstm.fit(x_train, y_train, epochs=30, batch_size=64, validation_data=(x_valid, y_valid), callbacks=[checkpoint], verbose=2)

Epoch 1/30
274/274 - 31s - loss: 3.7934 - categorical_accuracy: 0.4412 - val_loss: 3.2089 - val_categorical_accuracy: 0.4913

Epoch 00001: val_loss improved from inf to 3.20892, saving model to both_lstm.h5
Epoch 2/30
274/274 - 25s - loss: 2.8336 - categorical_accuracy: 0.5330 - val_loss: 2.7117 - val_categorical_accuracy: 0.5574

Epoch 00002: val_loss improved from 3.20892 to 2.71173, saving model to both_lstm.h5
Epoch 3/30
274/274 - 25s - loss: 2.2936 - categorical_accuracy: 0.5840 - val_loss: 2.4521 - val_categorical_accuracy: 0.5902

Epoch 00003: val_loss improved from 2.71173 to 2.45209, saving model to both_lstm.h5
Epoch 4/30
274/274 - 769s - loss: 1.8956 - categorical_accuracy: 0.6229 - val_loss: 2.3167 - val_categorical_accuracy: 0.6019

Epoch 00004: val_loss improved from 2.45209 to 2.31671, saving model to both_lstm.h5
Epoch 5/30
274/274 - 25s - loss: 1.5794 - categorical_accuracy: 0.6595 - val_loss: 2.2362 - val_categorical_accuracy: 0.6199

Epoch 00005: val_loss improved fr

In [48]:
# Saving the history of the Model-2

df_history_both_lstm = pd.DataFrame(history_both_lstm.history)
df_history_both_lstm
df_history_both_lstm.to_csv('df_history_both_lstm.csv')

In [49]:
# Loading the Model-2
both_lstm = load_model('both_lstm.h5')

In [50]:
# Model-2 training sequences
bleu_train_both_lstm = model_evaluation(both_lstm, eng_tokenizer, x_train, train)

French(Source) : appretetoi
Target : prepare yourself
Predicted : prepare yourself 

French(Source) : avezvous pris une douche
Target : have you showered
Predicted : have you showered 

French(Source) : rentre a la maison
Target : go home
Predicted : come home 

French(Source) : ils ont tort
Target : they are wrong
Predicted : they wrong 

French(Source) : aidemoi a sortir
Target : help me out
Predicted : get you out 

French(Source) : prenez les commandes
Target : take command
Predicted : look down 

French(Source) : restez ou vous etes
Target : hold it
Predicted : stay back 

French(Source) : je suis ruinee
Target : im ruined
Predicted : im wealthy 

French(Source) : je ne suis pas malheureux
Target : im not unhappy
Predicted : im not unhappy 

French(Source) : je peux y parvenir
Target : i can do it
Predicted : i can do it 

French(Source) : jaime tes jambes
Target : i like your legs
Predicted : i like your legs 

French(Source) : maitrisezvous
Target : control yourself
Predicted : 

In [51]:
# Testing the Model-2 test sequences
bleu_test_both_lstm = model_evaluation(both_lstm, eng_tokenizer, x_test, test)

French(Source) : jai oublie le livre
Target : i forgot the book
Predicted : i forgot the book 

French(Source) : tom est sans domicile fixe
Target : toms homeless
Predicted : tom is free 

French(Source) : jaide tom
Target : i help tom
Predicted : well ignored it 

French(Source) : jai achete un cactus
Target : i bought a cactus
Predicted : i had a seizure 

French(Source) : cest plutot grand
Target : its quite large
Predicted : thats very big 

French(Source) : ouvrez le coffre
Target : open the safe
Predicted : open the safe 

French(Source) : il est trop confiant
Target : hes too trusting
Predicted : he is too 

French(Source) : il adore les jouets
Target : he loves toys
Predicted : he likes oranges 

French(Source) : estce que vous avez le temps
Target : do you have time
Predicted : do you time time 

French(Source) : jai presente mes excuses
Target : i have apologized
Predicted : i finally up 

French(Source) : il nous faut de leau
Target : we need water
Predicted : we got some wa

In [52]:
# Model 3:  Encoder GRU and Decoder LSTM cell

def gru_lstm(french_vocab, english_vocab, french_timesteps, english_timesteps, num_units):
	learning_rate = 1e-2
	model = Sequential()
	model.add(Embedding(french_vocab, num_units, input_length= french_timesteps, mask_zero=True))
	model.add(GRU(num_units))
	model.add(RepeatVector(english_timesteps))
	model.add(LSTM(num_units,return_sequences=True))
	model.add(TimeDistributed(Dense(english_vocab, activation='softmax')))
	return model

In [53]:
# Defining the Model-3

learning_rate = 1e-2
gru_lstm = gru_lstm(french_vocabulary_size, english_vocabulary_size, french_maximum_length, english_maximum_length, 256)
gru_lstm.compile(optimizer=Adam(learning_rate), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [54]:
# Summary of Model-3

print(gru_lstm.summary())
plot_model(gru_lstm, to_file='gru_lstm.png', show_shapes=True)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 12, 256)           2053376   
_________________________________________________________________
gru_2 (GRU)                  (None, 256)               394752    
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 5, 256)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 5, 256)            525312    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 5, 3949)           1014893   
Total params: 3,988,333
Trainable params: 3,988,333
Non-trainable params: 0
_________________________________________________________________
None
('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz

In [55]:
# Fitting the Model-3

filename = 'gru_lstm.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
history_gru_lstm = gru_lstm.fit(x_train, y_train, epochs=30, batch_size=64, validation_data=(x_valid, y_valid), callbacks=[checkpoint], verbose=2)

Epoch 1/30
274/274 - 36s - loss: 3.8447 - categorical_accuracy: 0.4401 - val_loss: 3.3390 - val_categorical_accuracy: 0.4846

Epoch 00001: val_loss improved from inf to 3.33901, saving model to gru_lstm.h5
Epoch 2/30
274/274 - 30s - loss: 2.9326 - categorical_accuracy: 0.5280 - val_loss: 2.8344 - val_categorical_accuracy: 0.5526

Epoch 00002: val_loss improved from 3.33901 to 2.83437, saving model to gru_lstm.h5
Epoch 3/30
274/274 - 29s - loss: 2.4225 - categorical_accuracy: 0.5791 - val_loss: 2.5735 - val_categorical_accuracy: 0.5853

Epoch 00003: val_loss improved from 2.83437 to 2.57354, saving model to gru_lstm.h5
Epoch 4/30
274/274 - 29s - loss: 2.0803 - categorical_accuracy: 0.6107 - val_loss: 2.4446 - val_categorical_accuracy: 0.6018

Epoch 00004: val_loss improved from 2.57354 to 2.44455, saving model to gru_lstm.h5
Epoch 5/30
274/274 - 31s - loss: 1.8117 - categorical_accuracy: 0.6365 - val_loss: 2.3609 - val_categorical_accuracy: 0.6097

Epoch 00005: val_loss improved from 2.

In [56]:
# Saving the history of the Model-3

df_history_gru_lstm = pd.DataFrame(history_both_lstm.history)
df_history_gru_lstm.to_csv('df_history_gru_lstm.csv')
print(df_history_gru_lstm)

        loss  categorical_accuracy  val_loss  val_categorical_accuracy
0   3.793427              0.441154  3.208919                   0.49128
1   2.833559              0.533040  2.711730                   0.55744
2   2.293599              0.584000  2.452090                   0.59016
3   1.895594              0.622914  2.316713                   0.60192
4   1.579440              0.659497  2.236204                   0.61992
5   1.334634              0.693417  2.175927                   0.62880
6   1.123589              0.726263  2.158596                   0.63616
7   0.972408              0.754206  2.154496                   0.64736
8   0.849356              0.779669  2.160844                   0.64736
9   0.746623              0.800469  2.180151                   0.64880
10  0.660448              0.818903  2.209576                   0.65040
11  0.609541              0.832423  2.229174                   0.65304
12  0.561103              0.841737  2.251234                   0.65488
13  0.

In [57]:
# Loading the Model-3

gru_lstm = load_model('gru_lstm.h5')

In [58]:
# Model-3 on the training sequences

bleu_train_gru_lstm = model_evaluation(gru_lstm, eng_tokenizer, x_train, train)

French(Source) : appretetoi
Target : prepare yourself
Predicted : prepare yourself 

French(Source) : avezvous pris une douche
Target : have you showered
Predicted : do you showered 

French(Source) : rentre a la maison
Target : go home
Predicted : come home 

French(Source) : ils ont tort
Target : they are wrong
Predicted : they seem 

French(Source) : aidemoi a sortir
Target : help me out
Predicted : help out 

French(Source) : prenez les commandes
Target : take command
Predicted : take the 

French(Source) : restez ou vous etes
Target : hold it
Predicted : stay stay 

French(Source) : je suis ruinee
Target : im ruined
Predicted : im brave 

French(Source) : je ne suis pas malheureux
Target : im not unhappy
Predicted : im not unhappy 

French(Source) : je peux y parvenir
Target : i can do it
Predicted : i can handle it it 

French(Source) : jaime tes jambes
Target : i like your legs
Predicted : i like your legs 

French(Source) : maitrisezvous
Target : control yourself
Predicted : hi

In [59]:
# Testing the Model-3 on the test sequences

bleu_test_gru_lstm = model_evaluation(gru_lstm, eng_tokenizer, x_test, test)

French(Source) : jai oublie le livre
Target : i forgot the book
Predicted : i got the book 

French(Source) : tom est sans domicile fixe
Target : toms homeless
Predicted : tom is early 

French(Source) : jaide tom
Target : i help tom
Predicted : ill call 

French(Source) : jai achete un cactus
Target : i bought a cactus
Predicted : i have a fortune 

French(Source) : cest plutot grand
Target : its quite large
Predicted : thats is big 

French(Source) : ouvrez le coffre
Target : open the safe
Predicted : open the 

French(Source) : il est trop confiant
Target : hes too trusting
Predicted : hes is too 

French(Source) : il adore les jouets
Target : he loves toys
Predicted : he likes himself 

French(Source) : estce que vous avez le temps
Target : do you have time
Predicted : is you count it 

French(Source) : jai presente mes excuses
Target : i have apologized
Predicted : i my my 

French(Source) : il nous faut de leau
Target : we need water
Predicted : we need some water 

French(Source

In [60]:
# Model 4:  Encoder LSTM and Decoder GRU cell

def lstm_gru(french_vocab, english_vocab, french_timesteps, english_timesteps, num_units):
	learning_rate = 1e-2
	model = Sequential()
	model.add(Embedding(french_vocab, num_units, input_length= french_timesteps, mask_zero=True))
	model.add(LSTM(num_units))
	model.add(RepeatVector(english_timesteps))
	model.add(GRU(num_units,return_sequences=True))
	model.add(TimeDistributed(Dense(english_vocab, activation='softmax')))
	return model

In [61]:
# Defining the Model-4

learning_rate = 1e-2
lstm_gru = lstm_gru(french_vocabulary_size, english_vocabulary_size, french_maximum_length, english_maximum_length, 256)
lstm_gru.compile(optimizer=Adam(learning_rate), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [62]:
# Summary of Model-4

print(lstm_gru.summary())
plot_model(lstm_gru, to_file='lstm_gru.png', show_shapes=True)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 12, 256)           2053376   
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               525312    
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 5, 256)            0         
_________________________________________________________________
gru_3 (GRU)                  (None, 5, 256)            394752    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 5, 3949)           1014893   
Total params: 3,988,333
Trainable params: 3,988,333
Non-trainable params: 0
_________________________________________________________________
None
('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz

In [63]:
# Fitting the Model-4

filename = 'lstm_gru.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
history_lstm_gru = lstm_gru.fit(x_train, y_train, epochs=30, batch_size=64, validation_data=(x_valid, y_valid), callbacks=[checkpoint], verbose=2)

Epoch 1/30
274/274 - 30s - loss: 3.9620 - categorical_accuracy: 0.4267 - val_loss: 3.4909 - val_categorical_accuracy: 0.4603

Epoch 00001: val_loss improved from inf to 3.49087, saving model to lstm_gru.h5
Epoch 2/30
274/274 - 29s - loss: 3.1797 - categorical_accuracy: 0.4939 - val_loss: 3.0734 - val_categorical_accuracy: 0.5124

Epoch 00002: val_loss improved from 3.49087 to 3.07341, saving model to lstm_gru.h5
Epoch 3/30
274/274 - 25s - loss: 2.6673 - categorical_accuracy: 0.5445 - val_loss: 2.7395 - val_categorical_accuracy: 0.5551

Epoch 00003: val_loss improved from 3.07341 to 2.73948, saving model to lstm_gru.h5
Epoch 4/30
274/274 - 25s - loss: 2.2402 - categorical_accuracy: 0.5841 - val_loss: 2.5325 - val_categorical_accuracy: 0.5758

Epoch 00004: val_loss improved from 2.73948 to 2.53248, saving model to lstm_gru.h5
Epoch 5/30
274/274 - 28s - loss: 1.8911 - categorical_accuracy: 0.6190 - val_loss: 2.3928 - val_categorical_accuracy: 0.5910

Epoch 00005: val_loss improved from 2.

In [64]:
# Saving the history of the Model-4

df_history_lstm_gru = pd.DataFrame(history_both_lstm.history)
df_history_lstm_gru.to_csv('df_history_lstm_gru.csv')
print(df_history_lstm_gru)

        loss  categorical_accuracy  val_loss  val_categorical_accuracy
0   3.793427              0.441154  3.208919                   0.49128
1   2.833559              0.533040  2.711730                   0.55744
2   2.293599              0.584000  2.452090                   0.59016
3   1.895594              0.622914  2.316713                   0.60192
4   1.579440              0.659497  2.236204                   0.61992
5   1.334634              0.693417  2.175927                   0.62880
6   1.123589              0.726263  2.158596                   0.63616
7   0.972408              0.754206  2.154496                   0.64736
8   0.849356              0.779669  2.160844                   0.64736
9   0.746623              0.800469  2.180151                   0.64880
10  0.660448              0.818903  2.209576                   0.65040
11  0.609541              0.832423  2.229174                   0.65304
12  0.561103              0.841737  2.251234                   0.65488
13  0.

In [65]:
# Loading Model-4
lstm_gru = load_model('lstm_gru.h5')

In [66]:
# Model-4 on training sequences

bleu_train_lstm_gru = model_evaluation(lstm_gru, eng_tokenizer, x_train, train)

French(Source) : appretetoi
Target : prepare yourself
Predicted : prepare down 

French(Source) : avezvous pris une douche
Target : have you showered
Predicted : you you showered 

French(Source) : rentre a la maison
Target : go home
Predicted : come home home 

French(Source) : ils ont tort
Target : they are wrong
Predicted : they all wrong 

French(Source) : aidemoi a sortir
Target : help me out
Predicted : let me out 

French(Source) : prenez les commandes
Target : take command
Predicted : take control 

French(Source) : restez ou vous etes
Target : hold it
Predicted : turn home 

French(Source) : je suis ruinee
Target : im ruined
Predicted : im rested 

French(Source) : je ne suis pas malheureux
Target : im not unhappy
Predicted : im not unhappy 

French(Source) : je peux y parvenir
Target : i can do it
Predicted : i can do it 

French(Source) : jaime tes jambes
Target : i like your legs
Predicted : i love your legs 

French(Source) : maitrisezvous
Target : control yourself
Predict

In [67]:
# Testing Model-4 on test sequences

bleu_test_lstm_gru = model_evaluation(lstm_gru, eng_tokenizer, x_test, test)

French(Source) : jai oublie le livre
Target : i forgot the book
Predicted : i wrote the book 

French(Source) : tom est sans domicile fixe
Target : toms homeless
Predicted : tom is overjoyed 

French(Source) : jaide tom
Target : i help tom
Predicted : describe tom 

French(Source) : jai achete un cactus
Target : i bought a cactus
Predicted : i have a grenade 

French(Source) : cest plutot grand
Target : its quite large
Predicted : its too large 

French(Source) : ouvrez le coffre
Target : open the safe
Predicted : lock the safe 

French(Source) : il est trop confiant
Target : hes too trusting
Predicted : hes too cloudy 

French(Source) : il adore les jouets
Target : he loves toys
Predicted : he sold his 

French(Source) : estce que vous avez le temps
Target : do you have time
Predicted : do you wasting 

French(Source) : jai presente mes excuses
Target : i have apologized
Predicted : i just an 

French(Source) : il nous faut de leau
Target : we need water
Predicted : i need more water 